# Use SPSS to predict customer churn with `ibm-watsonx-ai`

This notebook contains steps to deploy sample SPSS stream, and start scoring new data. 

Some familiarity with bash is helpful. This notebook uses Python 3.12.

## Learning goals

The learning goals of this notebook are:

-  Working with the watsonx.ai instance
-  Online deployment of SPSS model
-  Scoring data using deployed model


## Contents

This notebook contains the following parts:

1. [Setup](#setup)
2. [Upload model](#upload) 
3. [Deploy model](#deploy)
4. [Scoring](#score)
5. [Clean up](#cleanup)
6. [Summary and next steps](#summary)

<a id="setup"></a>
## 1. Set up the environment

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Contact with your Cloud Pak for Data administrator and ask them for your account credentials

### Install dependencies
**Note:** `ibm-watsonx-ai` documentation can be found <a href="https://ibm.github.io/watsonx-ai-python-sdk/index.html" target="_blank" rel="noopener no referrer">here</a>.

In [1]:
%pip install -U wget | tail -n 1
%pip install -U ibm-watsonx-ai | tail -n 1

#### Define credentials

Authenticate the watsonx.ai Runtime service on IBM Cloud Pak for Data. You need to provide the **admin's** `username` and the platform `url`.

In [ ]:
username = "PASTE YOUR USERNAME HERE"
url = "PASTE THE PLATFORM URL HERE"

Use the **admin's** `api_key` to authenticate watsonx.ai Runtime services:

In [ ]:
import getpass
from ibm_watsonx_ai import Credentials

credentials = Credentials(
    username=username,
    api_key=getpass.getpass("Enter your watsonx.ai API key and hit enter: "),
    url=url,
    instance_id="openshift",
    version="5.2",
)

Alternatively you can use the **admin's** `password`:

In [3]:
import getpass
from ibm_watsonx_ai import Credentials

if "credentials" not in locals() or not credentials.api_key:
    credentials = Credentials(
        username=username,
        password=getpass.getpass("Enter your watsonx.ai password and hit enter: "),
        url=url,
        instance_id="openshift",
        version="5.2",
    )

#### Create `APIClient` instance

In [4]:
from ibm_watsonx_ai import APIClient

client = APIClient(credentials)

### Working with spaces

First of all, you need to create a space that will be used for your work. If you do not have space already created, you can use `{PLATFORM_URL}/ml-runtime/spaces?context=icp4data` to create one.

- Click New Deployment Space
- Create an empty space
- Go to space `Settings` tab
- Copy `space_id` and paste it below

**Tip**: You can also use SDK to prepare the space for your work. More information can be found [here](https://github.com/IBM/watsonx-ai-samples/blob/master/cpd5.2/notebooks/python_sdk/instance-management/Space%20management.ipynb).

**Action**: Assign space ID below

In [ ]:
space_id = "PASTE YOUR SPACE ID HERE"

You can use `list` method to print all existing spaces.

In [ ]:
client.spaces.list(limit=10)

To be able to interact with all resources available in watsonx.ai, you need to set **space** which you will be using.

In [6]:
client.set.default_space(space_id)

'SUCCESS'

<a id="upload"></a>
## 2. Upload model

In this section you will learn how to upload the model to the Cloud.

**Action**: Download sample SPSS model from Git repository using `wget`.

In [7]:
import os
import wget

sample_dir = "spss_sample_model"
if not os.path.isdir(sample_dir):
    os.mkdir(sample_dir)

filename = os.path.join(sample_dir, "customer-satisfaction-prediction.str")
if not os.path.isfile(filename):
    filename = wget.download(
        "https://github.com/IBM/watsonx-ai-samples/raw/master/cpd5.2/models/spss/customer_satisfaction/model/customer-satisfaction-prediction.str",
        out=sample_dir,
    )
print(filename)

spss_sample_model/customer-satisfaction-prediction.str


List available software specifications

In [8]:
client.software_specifications.list()

,NAME,ID,TYPE,STATE,REPLACEMENT
0,autoai-kb_rt25.1-py3.12,046f5252-0d07-5c23-81ef-18a1a31cdce3,base,supported,
1,tensorflow_rt25.1-py3.12-edt,0747503b-cc2e-5249-9019-59df39a112b8,base,supported,
2,watsonx-cfm-caikit-1.0,0cee3c55-472f-57b1-84bd-72f5d066dbe4,base,supported,
3,watsonx-textgen-fm-1.0,129aec82-7e65-5c78-b812-4c0a74b916f5,base,supported,
4,masking-flows-spark,13666829-5570-53a7-927b-52d42a101d93,base,supported,
5,autoai-ts_rt25.1-py3.12,1c0c9e1d-8523-520f-867e-102173f16f5e,base,supported,
6,tensorflow_rt24.1-py3.11,2c33167d-b11c-5490-a305-3e5e95db5c4d,base,supported,
7,pytorch-onnx_rt24.1-py3.11,2da185aa-eac3-59a5-bb8e-0e5b60458a15,base,supported,
8,tensorflow_rt24.1-py3.11-edt,2f53b747-c01a-58a6-a20c-8390ecfd8625,base,supported,
9,kernel-spark3.5-py3.11,30ad99e2-7cfb-511a-9a3e-4cc308071fd3,base,supported,


Store SPSS model in your watsonx.ai instance.

In [9]:
sw_spec_id = client.software_specifications.get_id_by_name("spss-modeler_18.2")

model_meta_props = {
    client.repository.ModelMetaNames.NAME: "SPSS customer satisfaction model",
    client.repository.ModelMetaNames.TYPE: "spss-modeler_18.2",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_ID: sw_spec_id,
}

model_details = client.repository.store_model(filename, model_meta_props)

**Note:** You can see that model is successfully stored in watsonx.ai.

In [10]:
client.repository.list_models()

,ID,NAME,CREATED,TYPE,SPEC_STATE,SPEC_REPLACEMENT
0,a299af8d-73d2-4ac9-ba3d-9a3fc239ba08,SPSS customer satisfaction model,2025-05-16T09:02:10Z,spss-modeler_18.2,supported,


<a id="deploy"></a>
## 3. Create online deployment
You can use commands bellow to create online deployment for stored model (web service).

In [11]:
model_id = client.repository.get_model_id(model_details)

deployment = client.deployments.create(
    artifact_id=model_id,
    meta_props={
        client.deployments.ConfigurationMetaNames.NAME: "SPSS deployment",
        client.deployments.ConfigurationMetaNames.ONLINE: {},
    },
)



######################################################################################

Synchronous deployment creation for id: 'a299af8d-73d2-4ac9-ba3d-9a3fc239ba08' started

######################################################################################


initializing
Note: online_url is deprecated and will be removed in a future release. Use serving_urls instead.
.....................................
ready


-----------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_id='bf691aa7-82db-4324-8070-232ac7306de8'
-----------------------------------------------------------------------------------------------




<a id="score"></a>
## 4. Scoring

You can send new scoring records to web-service deployment using `score` method.

In [12]:
deployment_id = client.deployments.get_id(deployment)

scoring_data = {
    client.deployments.ScoringMetaNames.INPUT_DATA: [
        {
            "fields": [
                "customerID",
                "gender",
                "SeniorCitizen",
                "Partner",
                "Dependents",
                "tenure",
                "PhoneService",
                "MultipleLines",
                "InternetService",
                "OnlineSecurity",
                "OnlineBackup",
                "DeviceProtection",
                "TechSupport",
                "StreamingTV",
                "StreamingMovies",
                "Contract",
                "PaperlessBilling",
                "PaymentMethod",
                "MonthlyCharges",
                "TotalCharges",
                "Churn",
                "SampleWeight",
            ],
            "values": [
                [
                    "3638-WEABW",
                    "Female",
                    0,
                    "Yes",
                    "No",
                    58,
                    "Yes",
                    "Yes",
                    "DSL",
                    "No",
                    "Yes",
                    "No",
                    "Yes",
                    "No",
                    "No",
                    "Two year",
                    "Yes",
                    "Credit card (automatic)",
                    59.9,
                    3505.1,
                    "No",
                    2.768,
                ]
            ],
        }
    ]
}

predictions = client.deployments.score(deployment_id, scoring_data)
print(predictions)

{'predictions': [{'fields': ['customerID', 'Churn', 'Predicted Churn', 'Probability of Churn'], 'values': [['3638-WEABW', 'No', 'No', 0.0526309571556145]]}]}


As we can see this sample telco customer is satisfied ("Predicted Churn", "No").

<a id="cleanup"></a>
## 5. Clean up 

If you want to clean up all created assets:
- experiments
- trainings
- pipelines
- model definitions
- models
- functions
- deployments

please follow up this sample [notebook](https://github.com/IBM/watsonx-ai-samples/blob/master/cpd5.2/notebooks/python_sdk/instance-management/Machine%20Learning%20artifacts%20management.ipynb).

<a id="summary"></a>
## 6. Summary and next steps     

You successfully completed this notebook! You learned how to use watsonx.ai for SPSS model deployment and scoring. 
 
Check out our _<a href="https://ibm.github.io/watsonx-ai-python-sdk/samples.html" target="_blank" rel="noopener no referrer">Online Documentation</a>_ for more samples, tutorials, documentation, how-tos, and blog posts. 

### Author

**Lukasz Cmielowski, PhD,** is an Automation Architect and Data Scientist at IBM with a track record of developing enterprise-level applications that substantially increases clients' ability to turn data into actionable knowledge.

Copyright © 2020-2025 IBM. This notebook and its source code are released under the terms of the MIT License.